In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/review/reviews_v1_hiring_task.csv


This is an failed expt was trying to build on top of tranformer model and do the training and prediction. but while training got an error which I could not resolve.Thats why used the tradional ml algoithms which is found in the other file under the folder 'notebooks'

COMPLETE PART-A assignment 

In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from termcolor import colored
import torch 
import torch.nn as nn
import torch.optim as optim
from torchtext.vocab import FastText, vocab
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.nn.utils.clip_grad import clip_grad_norm_
from torch.utils.data import TensorDataset, DataLoader
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from gensim.models.phrases import Phrases, Phraser
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TreebankWordTokenizer
from string import punctuation
import re, random, pathlib
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()

from transformers import RobertaModel, RobertaTokenizer, get_linear_schedule_with_warmup, AutoTokenizer, AutoModel

nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('vader_lexicon')
nltk.download('punkt')


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

path_files = pathlib.Path('./')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/kaggle/input/review/reviews_v1_hiring_task.csv


In [10]:
data.shape


NameError: name 'data' is not defined

In [14]:
data    = pd.read_csv('/kaggle/input/review/reviews_v1_hiring_task.csv', sep=',', encoding='latin-1').sample(500)
data.head()

,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
1327,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-02-20T00:00:00.000Z,2017-04-03T19:50:50Z,"2017-04-03T03:02:00.000Z,2017-04-30T00:42:00.0...",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,I bought this Kindle as a gift for my mother s...,"Excellent tablet for email, Facebook and media.",NaN,NaN,Robbie
1358,AVqVGWLKnnc1JgDc3jF1,"Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16...",B018Y23MNM,Amazon,"Tablets,Fire Tablets,Computers & Tablets,All T...",firekidseditiontablet7displaywifi16gbgreenkidp...,Amazon,2017-01-16T00:00:00.000Z,2017-06-21T07:37:47Z,"2017-06-04T02:16:38.290Z,2017-06-03T18:41:12.851Z",...,False,NaN,0.0,4.0,http://reviews.bestbuy.com/3545/5026300/review...,I love it it comes with a 12 month kids subscr...,It a great Tablet,NaN,NaN,Cata
665,AV1YnRtnglJLPUi8IJmV,"Kindle Voyage E-reader, 6 High-Resolution Disp...",B00OQVZDJM,Amazon,"Walmart for Business,Office Electronics,Tablet...","amazon/b00oqvzdjm,848719056099,amazonkindlepap...",Amazon,2017-01-17T00:00:00.000Z,2017-09-05T22:09:30Z,"2017-08-31T22:32:41.801Z,2017-08-02T19:53:23.2...",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/9439005/review...,"Gift for spouse, easily downloaded books. Uses...",Excellent,NaN,NaN,Outsider1
772,AV1YnRtnglJLPUi8IJmV,Amazon Kindle Paperwhite - eBook reader - 4 GB...,B00OQVZDJM,Amazon,"Walmart for Business,Office Electronics,Tablet...","amazon/b00oqvzdjm,848719056099,amazonkindlepap...",Amazon,2017-02-21T00:00:00.000Z,2017-09-05T22:09:30Z,"2017-08-31T22:33:22.863Z,2017-08-02T19:54:37.1...",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/9439005/review...,Told myself I wouldn't buy another e-reader un...,Great Product,NaN,NaN,RamiLope
1400,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-03-12T00:00:00.000Z,2017-05-21T05:55:39Z,"2017-04-30T00:42:00.000Z,2017-06-07T09:03:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,"I have only used this a few times so far, but ...",Like the product,NaN,NaN,ST19


In [15]:
data.shape

(500, 21)

Feature Engineering

In [16]:
Notkeep = [d for d in data.columns if d not in ['reviews.text','reviews.text','reviews.rating','manufacturer','reviews.date','name','id','brand','categories']]
data = data.drop(columns=Notkeep)
data

,id,name,brand,categories,manufacturer,reviews.date,reviews.rating,reviews.text
1327,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",Amazon,2017-02-20T00:00:00.000Z,5.0,I bought this Kindle as a gift for my mother s...
1358,AVqVGWLKnnc1JgDc3jF1,"Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16...",Amazon,"Tablets,Fire Tablets,Computers & Tablets,All T...",Amazon,2017-01-16T00:00:00.000Z,4.0,I love it it comes with a 12 month kids subscr...
665,AV1YnRtnglJLPUi8IJmV,"Kindle Voyage E-reader, 6 High-Resolution Disp...",Amazon,"Walmart for Business,Office Electronics,Tablet...",Amazon,2017-01-17T00:00:00.000Z,5.0,"Gift for spouse, easily downloaded books. Uses..."
772,AV1YnRtnglJLPUi8IJmV,Amazon Kindle Paperwhite - eBook reader - 4 GB...,Amazon,"Walmart for Business,Office Electronics,Tablet...",Amazon,2017-02-21T00:00:00.000Z,5.0,Told myself I wouldn't buy another e-reader un...
1400,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",Amazon,2017-03-12T00:00:00.000Z,5.0,"I have only used this a few times so far, but ..."
...,...,...,...,...,...,...,...,...
227,AVqVGWLKnnc1JgDc3jF1,"Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16...",Amazon,"Tablets,Fire Tablets,Computers & Tablets,All T...",Amazon,2017-01-02T00:00:00.000Z,2.0,I bought 2 Amazon Kid's Fire Tablets for my tw...
1160,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",Amazon,2017-01-28T00:00:00.000Z,5.0,"Easy, fast and efficient.great lil tablet in t..."
1407,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",Amazon,2017-01-04T00:00:00.000Z,4.0,My grandmother enjoys this tablet ... it use a...
344,AV1YnRtnglJLPUi8IJmV,Amazon Kindle Paperwhite - eBook reader - 4 GB...,Amazon,"Walmart for Business,Office Electronics,Tablet...",Amazon,2016-02-26T00:00:00.000Z,2.0,Very nice eReader - a big step up from the bas...


Since we need numerical data and we have currently categorical data ,droppig duplicated and missing values

Preprocessinng text

In [ ]:
stemmer = WordNetLemmatizer()


nltk.download('stopwords')
en_stop       = set(nltk.corpus.stopwords.words('english'))
to_be_removed = list(en_stop) + list(punctuation)

# Preprocess text for transformers
def preprocess_text(data, full_process=True):
        # Remove all the special characters
        data = re.sub(r'\W', ' ', str(data))
        # remove all single characters
        data= re.sub(r'\s+[a-zA-Z]\s+', ' ', data)
        # Remove single characters from the start
        data = re.sub(r'\^[a-zA-Z]\s+', ' ', data)
        # Substituting multiple spaces with single space
        data = re.sub(r'\s+', ' ', data, flags=re.I)
        # Removing prefixed 'b'
        data= re.sub(r'^b\s+', '', data)
        # Converting to Lowercase
        data = data.lower()
        if full_process:
            # Lemmatization
            tokens = data.split()
            tokens = [stemmer.lemmatize(word) for word in tokens]
            tokens = [word for word in tokens if word not in en_stop]
            tokens = [word for word in tokens if len(word) > 3]
            data = ' '.join(tokens)
        return data

In [17]:
data=preprocess_text(data)
types_str = data.select_dtypes(include='object').columns
types_int = data.select_dtypes(include=[int,float]).columns
data[types_str] = data[types_str].fillna('NA')
data[types_int] = SimpleImputer(strategy='mean').fit_transform(data[types_int])
# Drop Duplicates
duplicate = data[['reviews.rating', 'reviews.text','id']].duplicated()
data      = data.loc[duplicate==False]
data = data.reset_index()
print('Drop Duplicated:',duplicate.sum())

# Final frame
cols = ['reviews.rating', 'reviews.text','id']
df = data[cols]
df.fillna(' ',inplace=True)
df['reviews.text'].astype(str)
df['reviews.rating'].astype(str)
df['id'].astype(str)

Drop Duplicated: 0


/tmp/ipykernel_34/3654791195.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna(' ',inplace=True)


0      AVqkIhwDv8e3D1O-lebb
1      AVqVGWLKnnc1JgDc3jF1
2      AV1YnRtnglJLPUi8IJmV
3      AV1YnRtnglJLPUi8IJmV
4      AVqkIhwDv8e3D1O-lebb
               ...         
495    AVqVGWLKnnc1JgDc3jF1
496    AVqkIhwDv8e3D1O-lebb
497    AVqkIhwDv8e3D1O-lebb
498    AV1YnRtnglJLPUi8IJmV
499    AVqkIhwDv8e3D1O-lebb
Name: id, Length: 500, dtype: object

In [21]:
df = data[['reviews.text','reviews.rating','reviews.date','name','manufacturer']]
df #getting the clean data

,reviews.text,reviews.rating,reviews.date,name,manufacturer
0,I bought this Kindle as a gift for my mother s...,5.0,2017-02-20T00:00:00.000Z,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",Amazon
1,I love it it comes with a 12 month kids subscr...,4.0,2017-01-16T00:00:00.000Z,"Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16...",Amazon
2,"Gift for spouse, easily downloaded books. Uses...",5.0,2017-01-17T00:00:00.000Z,"Kindle Voyage E-reader, 6 High-Resolution Disp...",Amazon
3,Told myself I wouldn't buy another e-reader un...,5.0,2017-02-21T00:00:00.000Z,Amazon Kindle Paperwhite - eBook reader - 4 GB...,Amazon
4,"I have only used this a few times so far, but ...",5.0,2017-03-12T00:00:00.000Z,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",Amazon
...,...,...,...,...,...
495,I bought 2 Amazon Kid's Fire Tablets for my tw...,2.0,2017-01-02T00:00:00.000Z,"Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16...",Amazon
496,"Easy, fast and efficient.great lil tablet in t...",5.0,2017-01-28T00:00:00.000Z,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",Amazon
497,My grandmother enjoys this tablet ... it use a...,4.0,2017-01-04T00:00:00.000Z,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",Amazon
498,Very nice eReader - a big step up from the bas...,2.0,2016-02-26T00:00:00.000Z,Amazon Kindle Paperwhite - eBook reader - 4 GB...,Amazon



Preprocessing for Transformers

Since the models need inputs of equal length, we need to pad sequences tokenized sentences. We provide the maximum length based on the larges size of headline. On top of the word ids, the model requires an attention mask to indicate where the input sentences are padded or not.


In [22]:


df.head()



,reviews.text,reviews.rating,reviews.date,name,manufacturer
0,I bought this Kindle as a gift for my mother s...,5.0,2017-02-20T00:00:00.000Z,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",Amazon
1,I love it it comes with a 12 month kids subscr...,4.0,2017-01-16T00:00:00.000Z,"Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16...",Amazon
2,"Gift for spouse, easily downloaded books. Uses...",5.0,2017-01-17T00:00:00.000Z,"Kindle Voyage E-reader, 6 High-Resolution Disp...",Amazon
3,Told myself I wouldn't buy another e-reader un...,5.0,2017-02-21T00:00:00.000Z,Amazon Kindle Paperwhite - eBook reader - 4 GB...,Amazon
4,"I have only used this a few times so far, but ...",5.0,2017-03-12T00:00:00.000Z,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",Amazon


For training and testing the cleaned data

In [26]:


#split train test
from sklearn.model_selection import train_test_split


X=df['reviews.text']
y=df['reviews.rating']




Optimization Settings

We use Adam gradient descent that combines decaying averages of past and past squared gradients and generally offers good performances. We add a linear scheduler for the learning rate to smooth the optimization process.


X = train['Cleared Review'].apply(lambda text: ' '.join(text))
Y = train['Score']

We will be using traditional ML algorithms Descion Tree ALgorithm and logistic regression,Random forest regressor

In [27]:
models_metrics = pd.DataFrame({'Name':['LinReg', 'DesTree', 'RanFor'],
                               'Accuracy':[0, 0, 0],
                               'RMSE':[0, 0, 0],
                               'R^2':[0, 0, 0]})

models_metrics.set_index('Name', inplace=True)

In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spacy
import re
import string
import seaborn as sns
import nltk 
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import confusion_matrix
from collections import Counter

In [42]:
def get_report(test_data, predicted_data, model_name=''):
    metrics = [mean_squared_error(test_data, predicted_data, squared=False),
              r2_score(test_data, predicted_data),
              accuracy_score(test_data, predicted_data)]
    if model_name:
        models_metrics.at[model_name, 'RMSE'] = metrics[0]
        models_metrics.at[model_name, 'Accuracy'] = metrics[2]
        models_metrics.at[model_name, 'R^2'] = metrics[1]
    print(f'The RMSE on data is: {metrics[0]}')
    print(f'The R2 on data is: {metrics[1]}')
    print(f'Accuracy is: {metrics[2]}')

In [43]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Assuming X and y are your original data
# X, y = ... 

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply one-hot encoding only to the feature variables
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

# Align columns of X_test to match X_train
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# Ensure y_train and y_test are 1-dimensional
y_train = y_train if y_train.ndim == 1 else y_train.iloc[:, 0]
y_test = y_test if y_test.ndim == 1 else y_test.iloc[:, 0]

# Initialize and fit the logistic regression model
log_model = LogisticRegression()
log_model.fit(X_train, y_train)

# Make predictions on the test data
log_test_pred = log_model.predict(X_test)

log_train_pred = log_model.predict(X_train)

In [45]:
print("Metrics for test data:")
get_report(y_test, log_test_pred, 'LinReg')
print("\nMetrics for train data:")
get_report(y_train, log_train_pred)


Metrics for test data:
The RMSE on data is: 1.4422205101855958
The R2 on data is: -0.5133876600698484
Accuracy is: 0.59

Metrics for train data:
The RMSE on data is: 1.4798648586948742
The R2 on data is: -0.5974033078648409
Accuracy is: 0.54


In [46]:


def normalize_predictions(pred):
    new_pred = [i if i <= 5 else 5 for i in pred]
    new_pred = [i if i >= 1 else 1 for i in new_pred]
    new_pred = [round(i) for i in new_pred]
    return new_pred



In [ ]:


tree_model = DecisionTreeRegressor()
tree_model.fit(X_train, y_train)

tree_pred_test = normalize_predictions(tree_model.predict(X_test))
tree_pred_train = normalize_predictions(tree_model.predict(X_train))



In [ ]:
print("Metrics for test data:")
get_report(y_test, tree_pred_test, 'DesTree')
print("\nMetrics for train data:")
get_report(y_train, tree_pred_train)


In [ ]:
random_forest_model = RandomForestRegressor(max_features=100)
random_forest_model.fit(X_train, y_train)

forest_test_pred = normalize_predictions(random_forest_model.predict(X_test))
forest_train_pred = normalize_predictions(random_forest_model.predict(X_train))

In [ ]:

print("Metrics for test data:")
get_report(y_test, forest_test_pred, 'RanFor')
print("\nMetrics for train data:")
get_report(y_train, forest_train_pred)

In [ ]:
models_name = models_metrics.index.tolist()
plt.figure(figsize=(7,7))
plt.bar(models_name, models_metrics['RMSE'], label='RMSE')
plt.bar(models_name, models_metrics['Accuracy'], label='Accuracy')
plt.bar(models_name, models_metrics['R^2'], label='R^2')
plt.title('Metrics')
plt.legend()
plt.show()

In [47]:
tfidf = TfidfVectorizer(min_df=0.001)
tfidf_data = tfidf.fit_transform(X)

tfidf_data = pd.DataFrame(tfidf_data.toarray(), columns=tfidf.get_feature_names_out())
tfidf_data.head()


,10,100,105,10and,10th,12,128,15,16,180,...,yesterday,yet,yo,you,young,younger,youngest,your,youtube,yr
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
X_train_idf = tfidf_data.loc[X_train.index,:]
X_test_idf = tfidf_data.loc[X_test.index,:]

In [49]:


X_train_idf.shape, X_test_idf.shape



((400, 2119), (100, 2119))

In [50]:
log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train_idf, y_train)

idf_log_test = log_model.predict(X_test_idf)
idf_log_train = log_model.predict(X_train_idf)

In [ ]:
print("Metrics for test data")
get_report(y_test, idf_log_test, 'LinReg')
print("\nMetrics for train data")
get_report(y_train, idf_log_train)

In [ ]:
tree_model = DecisionTreeRegressor(max_depth=10)
tree_model.fit(X_train_idf, y_train)

idf_tree_test = normalize_predictions(tree_model.predict(X_test_idf))
idf_tree_train = normalize_predictions(tree_model.predict(X_train_idf))

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "max_depth": [3,5,10,20,50,70],
}

clf = DecisionTreeRegressor(random_state=42)
grid_cv = GridSearchCV(clf, param_grid, n_jobs=-1).fit(X_train_idf, y_train)

print("Param for GS", grid_cv.best_params_)
print("CV score for GS", grid_cv.best_score_)

In [ ]:


r2 = list()
x = [10, 20, 30, 50, 70, 100]

for depth in x:
    tree_model = DecisionTreeRegressor(max_depth=depth)
    tree_model.fit(X_train_idf, y_train)
    idf_tree_test = normalize_predictions(tree_model.predict(X_test_idf))
    r2.append(r2_score(y_test, idf_tree_test))



In [ ]:
plt.scatter(x, r2)
plt.plot(x, r2, '-')
plt.xlabel('Max depth')
plt.ylabel('R^2 score')
plt.title('Decision Tree')
plt.show()

In [ ]:
tree_model = DecisionTreeRegressor(max_depth=20)
tree_model.fit(X_train_idf, y_train)

idf_tree_test = normalize_predictions(tree_model.predict(X_test_idf))
idf_tree_train = normalize_predictions(tree_model.predict(X_train_idf))


In [ ]:
print("Metrics for test data")
get_report(y_test, idf_tree_test, 'DesTree')
print("\nMetrics for train data")
get_report(y_train, idf_tree_train)

In [ ]:
mse_test = list()
mse_train = list()
x = [20, 50, 100, 200, 300, 500]

for num in x:
    random_forest_model = RandomForestRegressor(max_features=num)
    random_forest_model.fit(X_train_idf, y_train)
    idf_forest_test = normalize_predictions(random_forest_model.predict(X_test_idf))
    idf_forest_train = normalize_predictions(random_forest_model.predict(X_train_idf))
    mse_test.append(mean_squared_error(y_test, idf_forest_test))
    mse_train.append(mean_squared_error(y_train, idf_forest_train))


In [ ]:
plt.figure(figsize=(5,7))
plt.scatter(x, mse_test)
plt.scatter(x, mse_train)
plt.plot(x, mse_test, '-', label='Test MSE')
plt.plot(x, mse_train, '-', label='Train MSE')
plt.xlabel('Max features')
plt.ylabel('MSE')
plt.title('Random forest')
plt.legend()
plt.show()


Contantains the standalone function to do the rating predictions

In [ ]:

random_forest_model = RandomForestRegressor(max_features=200)
random_forest_model.fit(X_train_idf, y_train)
idf_forest_test = normalize_predictions(random_forest_model.predict(X_test_idf))
idf_forest_train = normalize_predictions(random_forest_model.predict(X_train_idf))

print("Metrics for test data")
get_report(y_test, idf_forest_test, 'RanFor')
print("\nMetrics for test data")
get_report(y_train, idf_forest_train)

def predict_with_model(model, X_test_idf):
    predictions = model.predict(X_test_idf)
    idf_forest_test = normalize_predictions(random_forest_model.predict(X_test_idf))
    return idf_forest_test
if __name__ == "__main__":
    review = "This product exceeded my expectations. It's really amazing!"
    predicted_rating = predict_with_model(random_forest_model,review)
    print(f"Predicted Rating: {predicted_rating}")


In [ ]:


models_name = models_metrics.index.tolist()
plt.figure(figsize=(7,7))
plt.bar(models_name, models_metrics['RMSE'], label='RMSE')
plt.bar(models_name, models_metrics['Accuracy'], label='Accuracy')
plt.bar(models_name, models_metrics['R^2'], label='R^2')
plt.title('Metrics')
plt.legend()
plt.show()



In [ ]:
def plot_conf_matrix(cm1, cm2, model):
    labels = [1,2,3,4,5]
    fig, axes = plt.subplots(1, 2, figsize=(10, 4))
    plt.suptitle(model)

    # Plotting the first confusion matrix
    sns.heatmap(cm1, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels, ax=axes[0])
    axes[0].set_title('For Test data')
    axes[0].set_ylabel('Predicted')
    axes[0].set_xlabel('Actual')

    # Plotting the second confusion matrix
    sns.heatmap(cm2, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels, ax=axes[1])
    axes[1].set_title('For Train Data')
    axes[1].set_ylabel('Predicted')
    axes[1].set_xlabel('Actual')

    plt.tight_layout()
    plt.show()

In [ ]:


plot_conf_matrix(confusion_matrix(y_test, idf_log_test), confusion_matrix(y_train, idf_log_train), 'Logistic Regression')



In [ ]:


plot_conf_matrix(confusion_matrix(y_test, idf_tree_test), confusion_matrix(y_train, idf_tree_train), 'Decision Tree')



In [ ]:
plot_conf_matrix(confusion_matrix(y_test, idf_forest_test), confusion_matrix(y_train, idf_forest_train), 'Random Forest')

BUILDING THE LLM RAG PIPELINE


We are installing Faiss as a vector database

In [28]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 58.1 MB/s eta 0:00:00:00:0100:01


RAG PIPELINE

We are using pretained models from Hugging face.Using  bert-uncased since its an regressor model

In [29]:
from transformers import AutoTokenizer, AutoModel
import torch
import faiss

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = AutoModel.from_pretrained("google-bert/bert-base-uncased")

# Function to encode reviews
def encode_reviews(reviews):
    inputs = tokenizer(reviews, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.pooler_output

# Encode the reviews
review_embeddings = encode_reviews(df['reviews.text'].tolist())

# Build the index
index = faiss.IndexFlatL2(review_embeddings.shape[1])
index.add(review_embeddings.cpu().numpy())



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [34]:
query = "What did users like about product Amazon kindle"
query_embedding = encode_reviews([query])

# Perform the search
D, I = index.search(query_embedding.cpu().numpy(),k=1)  # Adjust k as needed

# Retrieve the results
results = df['reviews.text'].iloc[I[0]].to_dict()
print(results)


{83: 'So slow returning mine for something different any suggestions'}


In [35]:
k = 5
distances, indices = index.search(query_embedding.cpu().numpy(), k)

# Fetch the top-k reviews
top_reviews =  df['reviews.text'].iloc[indices[0]]
print(top_reviews)


83     So slow returning mine for something different...
236    This is a great amazon fire tablet for kids hi...
108    Works great prety fast operating system dosent...
128    it was great for reading but the lag time for ...
38     This our second paper white my husband loves t...
Name: reviews.text, dtype: object


In [66]:
!pip install openai==0.28


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached openai-0.28.0-py3-none-any.whl.metadata (13 kB)
Using cached openai-0.28.0-py3-none-any.whl (76 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.0.0
    Uninstalling openai-1.0.0:
      Successfully uninstalled openai-1.0.0


In [75]:
!openai tools # failed expt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


usage: openai tools [-h] {fine_tunes.prepare_data} ...

options:
  -h, --help            show this help message and exit

Tools:
  {fine_tunes.prepare_data}
                        Convenience client side tools


In [36]:
!pip install rank-bm25 # we will be using bm25 for summarizing and retrieval of the documents

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


We are using BM25 because 

   
        BM25 is built upon the principles of TF-IDF, which considers the frequency of terms in documents and the inverse frequency of terms across all documents. This helps in identifying terms that are more significant in distinguishing documents.
        
        BM25 incorporates a length normalization component, which adjusts the term frequency based on the document length. This helps in balancing the influence of term frequency in longer documents, ensuring that they do not have an unfair advantage over shorter documents.


        BM25 allows for the tuning of parameters k1k1​ (term frequency saturation parameter) and bb (length normalization parameter). This flexibility enables customization based on specific datasets or retrieval tasks, optimizing performance.

        The relevance scoring mechanism of BM25 is robust, providing a numerical score that reflects the relevance of each document to the query. This makes it easier to rank and retrieve the most pertinent documents efficiently.

   
        BM25 performs well across diverse types of document collections, including web pages, scientific articles, and news articles, making it versatile for different summarizing and retrieval contexts.

  
        BM25 is computationally efficient, making it suitable for large-scale document retrieval tasks where performance and speed are critical.

   
   

In [48]:
# Load model directly
from transformers import AutoTokenizer, AutoModel

tokenizer_BAAI = AutoTokenizer.from_pretrained("BAAI/llm-embedder")
model_BAAI = AutoModel.from_pretrained("BAAI/llm-embedder")

In [49]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer_MONOT5= AutoTokenizer.from_pretrained("castorini/monot5-base-msmarco")
model_MONOT5= AutoModelForSeq2SeqLM.from_pretrained("castorini/monot5-base-msmarco")

In [56]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
from transformers import AutoTokenizer, AutoModel # was trying to use llm/embedder to get the embedding but even after pasting the hugging face cli token it did not work
import torch

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("llm/embedder")
model1 = AutoModel.from_pretrained("llm/embedder")
texts= df['reviews.text']
# Function to get embeddings
def get_embeddings(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        embeddings = model1(**inputs).last_hidden_state.mean(dim=1)
    return embeddings


In [107]:
texts= df['reviews.text']
def summarize_texts(texts, bm25, n=5):
    summaries = []
    for text in texts:
        tokenized_sentences = [sentence.split(" ") for sentence in text.split(".")]
        bm25 = BM25Okapi(tokenized_sentences)
        summary = bm25.get_top_n(query.split(" "), tokenized_sentences, n=n)
        summaries.append(" ".join([" ".join(sent) for sent in summary]))
    return summaries


In [109]:
texts= df['reviews.text']
from rank_bm25 import BM25Okapi

# Tokenize your documents
tokenized_corpus = [doc.split(" ") for doc in texts]
bm25 = BM25Okapi(tokenized_corpus)

# Function to retrieve documents
def retrieve_documents(query, bm25, n=10):
    tokenized_query = query.split(" ")
    scores = bm25.get_scores(tokenized_query)
    top_n = scores.argsort()[-n:][::-1]
    return [texts[i] for i in top_n], scores[top_n]


In [127]:
from huggingface_hub import notebook_login
notebook_login()

In [140]:
#The Rag pipeline where we are calling all the above defined functions for best results
texts= df['reviews.text']
from transformers import AutoTokenizer, AutoModel
import torch

# Load the tokenizer and model

def RAG_pipeline(query, texts, bm25, summarizer):
    # Step 1: Embedding
 
    # Step 2: Initial Retrieval
    initial_docs, scores = retrieve_documents(query, bm25)
  
    # Step 4: Summarization
    summarized_docs = summarize_texts(texts, bm25)
    
    return summarized_docs

# Example usage
query = "What did users like about product AMAZON KINDLE"
result = RAG_pipeline(query, texts,bm25,bm25)
print(result)


['  Very satisfied  My mother uses hers for email and some light gaming and for watching Netflix as well  I have 3 kindle tablets in my home and they are used daily  She loves the ease of using it', 'I love it it comes with a 12 month kids subscription', '  Definitely recommend  Uses while waiting for kids to finish school activities Gift for spouse, easily downloaded books', "  It's lighter than carrying most books and it's non-glare and also back lit  I got this for Christmas and have loved everyday since Told myself I wouldn't buy another e-reader until it was water-proof", 'I have only used this a few times so far, but I like it   I ended up purchasing a second one for my fiancÃ©  Perfect screen size too  The lightweight and pretty easy to navigate', 'Being able to sign in and not have to do any setting up makes this device super user friendly!', 'My granddaughter is really enjoying this tablet and even though she is only three she is able to easily use it', " I also don't like tha